In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from mpltools import annotation
import numpy as np
import pandas

# Session 1: Forward mode differentiation and source transformation

## Motivation

From the abstract for the session:

> *Derivatives are at the heart of scientific programming. From the Jacobian matrices used to solve nonlinear systems to the gradient vectors used for optimisation methods, from the backpropagation operation in machine learning to the data assimilation methods used in weather forecasting, all of these techniques rely on derivative information. Differentiable programming (also known as automatic/algorithmic differentiation (AD)) provides a suite of tools for users to compute derivatives of quantities in their code without any manual encoding.*

<div class="alert alert-block alert-danger">
<b>TODO: elaborate, e.g., some examples</b>
</div>

## Learning objectives

In today's session we will:

* Get a brief history of automatic differentiation.
* Learn about *forward mode* and the *source transformation* approach.
* Try out the *Tapenade* source transformation AD tool applied to some test problems.
* Verify the code generated by Tapenade both manually and using the *Taylor test*.
* Learn about compressed and matrix-free approaches for sparse problems.
* Consider how different levels of abstraction can be appropriate for different problems.

## Preparations

#### Terminology

This course introduces the concept of *differentiable programming*, a.k.a. *automatic differentiation (AD)*, or *algorithmic differentiation*. We will use the acronym AD henceforth.

#### Notation

For a differentiable *mathematical* function $f:A\rightarrow\mathbb{R}$ with scalar input (i.e., a single value) from $A\subseteq\mathbb{R}$, we make use of both the Lagrange notation $f'(x)$ and Leibniz notation $\frac{\mathrm{d}f}{\mathrm{d}x}$ for its derivative.
We **do not** use the physics notation for derivatives, so if you ever see (e.g.) $\dot{x}$ then this is just a variable name, not the derivative of $x$.

Similarly, for $m\in\mathbb{N}$ dimensional, differentiable, vector-valued function $\mathbf{f}:A\rightarrow\mathbb{R}^m$ with scalar input, we have derivative notations $\mathbf{f}'(x)$ and $\frac{\mathrm{d}\mathbf{f}}{\mathrm{d}x}$.

For a differentiable function with vector input (i.e., multiple inputs), we use partial derivative notation. For example, if $f:\mathbb{R}^2\rightarrow\mathbb{R}$ is written as $f=f(x,y)$ then we have the partial derivatives $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial y}$ with respect to first and second components, respectively. We use
$$\nabla f=\left(\frac{\partial f}{\partial x_1},\dots,\frac{\partial f}{\partial x_m}\right)$$
to denote the vector of all such partial derivatives. Similarly for vector-valued functions with multiple inputs.

When it comes to derivatives in code, we use the `_d` notation, which is standard in the AD literature. Its meaning will be described in due course.

## History

* Origins of AD in 1950s.
* However, it found a wider audience in the 1980s, when it became more relevant thanks to advances in both computer power and modern programming languages.
* Forward mode (the subject of this session) was discovered by Wengert in 1964.
* Further developed by Griewank in the late 1980s.

<div class="alert alert-block alert-danger">
<b>TODO: images?</b>
</div>

## Idea

The idea of AD is to **treat a model as a sequence of elementary instructions** (e.g., addition, multiplication, exponentiation). Here a *model* could be a function or subroutine, code block, or a whole program. Elementary operations are well-understood and their derivatives are known. As such, the derivative of the whole model may be computed by composing the derivatives of each operation using the *chain rule*.

#### Recap on A-level maths: the Chain Rule

Consider two composable, differentiable (mathematical) functions, $f$ and $g$, with composition $h=f\circ g$. By definition, this means
$$h(x)=(f\circ g)(x)=g(f(x)).$$

Then the *chain rule* states that the derivative of $h$ may be computed in terms of the derivatives of $f$ and $g$ using the formula
$$h'(x)=(f\circ g)'(x)=(f\circ g')(x)\,f'(x)=g'(f(x))\,f'(x).$$

Equivalently, in Leibniz notation:
$$\frac{\mathrm{d}h}{\mathrm{d}x}=\frac{\mathrm{d}g}{\mathrm{d}f}\frac{\mathrm{d}f}{\mathrm{d}x}.$$

For variables with multiple arguments, the result is equivalent for each partial derivative, e.g.,
$$\frac{\partial h}{\partial x}=\frac{\partial g}{\partial f}\frac{\partial f}{\partial x}.$$

## $f$ \& $g$ example

Consider two functions acting on real numbers:
$$f(x,y)=xy$$
and
$$g(z)=(\sin(z),\cos(z)).$$
Here $f:\mathbb{R}^2\rightarrow\mathbb{R}$ takes two inputs and returns a single output, while $g:\mathbb{R}\rightarrow\mathbb{R}^2$ takes a single input and returns two outputs.

<div class="alert alert-block alert-info">
<b>Optional exercise</b>

Convince yourself that it is well defined for these functions may be composed in either order. (Although they won't necessarily give the same value!)

<b>Solution</b>

<details>
The image of $f$ is the set of all real numbers, so its image is the same as the domain of $g$ (i.e., $\text{im}(f)=\mathbb{R}=\text{dom}(g)$).

The image of $g$ is $[-1,1]^2=[-1,1]\times[-1,1]$ because $\sin$ and $\cos$ give values between -1 and 1. Since this is a subset of $\mathbb{R}^2$, the image of $g$ is a subset of the domain of $f$ (i.e., $\text{im}(g)\subset\text{dom}(f)$).
</details>
</div>

## $f$ \& $g$ example (continued)

Consider the composition $h=f\circ g:\mathbb{R}^2\rightarrow\mathbb{R}^2$. Then we have
$$h(x,y)=(f\circ g)(x,y)=g(f(x,y))=g(xy)=(\sin(xy),\cos(xy)).$$

For the derivative of each component,
$$
\frac{\partial f}{\partial x}=y,
\quad\frac{\partial f}{\partial y}=x,
\quad\frac{\partial g}{\partial z}=(\cos(z),-\sin(z)).
$$

<div class="alert alert-block alert-info">
<b>Exercise</b>

Use the chain rule to work out the derivatives of each of the outputs with respect to each of the inputs, i.e.,
$$
\frac{\partial h_1}{\partial x},
\quad\frac{\partial h_1}{\partial y},
\quad\frac{\partial h_2}{\partial x},
\quad\frac{\partial h_2}{\partial y},
$$
where $h(x,y)=(h_1(x,y),h_2(x,y))$.

<b>Solution</b>

<details>
$$
\frac{\partial h_1}{\partial x}=\frac{\partial g_1}{\partial f}\frac{\partial f}{\partial x}=\cos(z)y=y\cos(xy),
\quad\frac{\partial h_1}{\partial y}=\frac{\partial g_1}{\partial f}\frac{\partial f}{\partial y}=\cos(z)x=x\cos(xy),
$$
where $z=f(x,y)$ and
$$
\quad\frac{\partial h_2}{\partial x}=\frac{\partial g_2}{\partial f}\frac{\partial f}{\partial x}=\sin(z)y=y\sin(xy),
\quad\frac{\partial h_2}{\partial y}=\frac{\partial g_2}{\partial f}\frac{\partial f}{\partial y}=\sin(z)x=x\sin(xy).
$$  
</details>
</div>

## $f$ \& $g$ example: Directed Acyclic Graph

<div class="column" style="width: 50%;">
  Recall
  $$f(x,y)=xy$$
  and
  $$g(z)=(\sin(z),\cos(z)).$$

  We can visualise the composition in terms of a DAG. Here $\dot{x}$ and $\dot{y}$ denote *seeds* for the derivatives of the input variables $x$ and $y$, respectively. Think of these as inputs from outside of the part of the program that we are differentiating.
</div>
<div class="column" style="width: 50%;">
  <div style="text-align: center;">
  <img src="images/forward_AD_as_DAG.png" style="width: 50%;">
  <div><strong>Figure 1:</strong> Directed Acyclic Graph (DAG) for the mathematical functions in Example 1.</div>
</div>

<div class="alert alert-block alert-danger">
<b>TODOs: 1. switch notation to have $x$ and $y$ as inputs and then $h_1$ and $h_2$ as outputs. 2. why aren't the columns working?</b>
</div>

## Directional derivative, a.k.a. Jacobian-vector product (JVP)

Consider a vector-valued function $\mathbf{f}$ mapping from a subspace $A\subseteq\mathbb{R}^n$ into $\mathbb{R}^m$, for some $m,n\in\mathbb{N}$:
$$\mathbf{f}:A\rightarrow\mathbb R^m.$$

Given input $\mathbf{x}\in A$ and a *seed vector* $\dot{\mathbf{x}}\in\mathbb{R}^n$, forward mode AD allows us to compute the *action* (matrix-vector product)
$$\text{JVP}(\mathbf{f},\mathbf{x},\dot{\mathbf{x}}):=\nabla\mathbf{f}(\mathbf{x})\,\dot{\mathbf{x}}.$$
Again, think of the seed vector as being an input from outside of the part of the program being differentiated.

Here $\nabla\mathbf{f}$ is referred to as the *Jacobian* for the map, so the above is known as a *Jacobian-vector product (JVP)*.

<div class="alert alert-block alert-warning">
<b>Note</b>
The computation is <em>matrix-free</em>. We don't actually need the Jacobian when we compute this product.
</div>

## Approach 1: Source transformation

High level idea: Given some (code) function `f(x)` and a seed vector `x_d`, generate the code for the function `f_d(x, x_d)` for its (directional) derivative.

#### Evaluation of the source transformation approach:

* Clearly demonstrates the differentiation process.
* Static analysis, done ahead of time.
* Does not require a 'tape' (see later).
* Often the difficult part is hooking the differentiated code into the wider model/build system.
* Limited number of tools.

#### Source transformation tools:

* [Tapenade](https://tapenade.gitlabpages.inria.fr/userdoc/build/html/index.html) (C, Fortran, Julia*)
* [OpenAD](https://www.mcs.anl.gov/OpenAD) (Fortran)
* [TAF](http://fastopt.com/products/taf) (Fortran) [commerical]
* [PSyAD](https://psyclone-adjoint.readthedocs.io/en/stable)* (domain-specific)

*\*Work in progress*

## $f$ \& $g$ example: source transformation

Below we have the Fortran code for the example functions above, written as subroutines. You can find this in the repository at `session1/exercises/fg/f.f90` and `session1/exercises/fg/g.f90`, respectively.

```fortran
subroutine f(x, y, z)
  implicit none
  real, intent(in)  :: x, y
  real, intent(out) :: z
  z = x * y
end subroutine f
```

```fortran
subroutine g(z, v)
  implicit none
  real, intent(in)  :: z
  real, intent(out), dimension(2) :: v
  v = [sin(z), cos(z)]
end subroutine g
```

<div class="alert alert-block alert-info">
<b>Exercise</b>
    
1. Either [install Tapenade](https://tapenade.gitlabpages.inria.fr/tapenade/distrib/README.html)* or visit the [Tapenade web interface](http://tapenade.inria.fr:8080/tapenade/index.jsp).
2. Run `tapenade -h` to see all the options.
3. Apply Tapenade to each of these subroutines using its default setting, which will apply forward mode to compute the JVP for some seed vector.
4. Inspect the output files `f_d.f90` and `g_d.f90` and check they are as you expect.
5. Inspect the message files `f_d.msg` and `g_d.msg`.

*Note that you will need to install Java if you don't already have it installed.

<b>Solutions 2</b>
    
<details>

```sh
$ tapenade -h
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
 Builds a differentiated program.
 Usage: tapenade [options]* filenames
  options:
   -head, -root <proc>     set the differentiation root procedure(s)
                           See FAQ for refined invocation syntax, e.g.
                           independent and dependent arguments, multiple heads...
   -tangent, -d            differentiate in forward/tangent mode (default)
   -reverse, -b            differentiate in reverse/adjoint mode
   -vector, -multi         turn on "vector" mode (i.e. multi-directional)
   -specializeactivity <unit_names or %all%>  Allow for several activity patterns per routine
   -primal, -p             turn off differentiation. Show pointer destinations
   -output, -o <file>      put all generated code into a single <file>
   -splitoutputfiles       split generated code, one file per top unit
   -outputdirectory, -O <directory>  put all generated files in <directory> (default: .)
   -I <includePath>        add a new search path for include files
   -tgtvarname <str>       set extension for tangent variables  (default %d)
   -tgtfuncname <str>      set extension for tangent procedures (default %_d)
   -tgtmodulename <str>    set extension for tangent modules and types (default %_diff)
   -adjvarname <str>       set extension for adjoint variables  (default %b)
   -adjfuncname <str>      set extension for adjoint procedures (default %_b)
   -adjmodulename <str>    set extension for adjoint modules and types (default %_diff)
   -modulename <str>       set extension for tangent&adjoint modules and types (default %_diff)
   -inputlanguage <lang>   language of  input files (fortran, fortran90,
                           fortran95, or C)
   -outputlanguage <lang>  language of output files (fortran, fortran90,
                           fortran95, or C)
   -ext <file>             incorporate external library description <file>
   -nolib                  don't load standard libraries descriptions
   -i<n>                   count <n> bytes for an integer (default -i4)
   -r<n>                   count <n> bytes for a real (default -r4)
   -dr<n>                  count <n> bytes for a double real (default -dr8)
   -p<n>                   count <n> bytes for a pointer (default -p8)
   -fixinterface           don't use activity to filter user-given (in)dependent vars
   -noinclude              inline include files
   -debugTGT               insert instructions for debugging tangent mode
   -debugADJ               insert instructions for debugging adjoint mode
   -tracelevel <n>         set the level of detail of trace milestones
   -msglevel <n>           set the level of detail of error messages
   -msginfile              insert error messages in output files
   -dump <file>            write a dump <file>
   -html                   display results in a web browser
   -nooptim <str>          turn off optimization <str> (in {activity, difftypes,
                           diffarguments, stripprimalmodules, spareinit, splitdiff, 
                           mergediff, saveonlyused, tbr, snapshot, diffliveness,
                           deadcontrol, recomputeintermediates,
                           everyoptim}
   -version                display Tapenade version information
 Report bugs to <tapenade@inria.fr>.
```

</details>

<br>
<b>Solutions 3 and 4</b>
    
<details>

Running
```sh
$ cd session1/exercises/fg
$ ls
f.f90 g.f90
$ tapenade f.f90
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Command: Took subroutine f as default differentiation root
Diff-liveness analysis turned off
@@ Created ./f_d.f90 
@@ Created ./f_d.msg
$ cat f_d.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of f in forward (tangent) mode:
!   variations   of useful results: z
!   with respect to varying inputs: x y
!   RW status of diff variables: x:in y:in z:out
SUBROUTINE F_D(x, xd, y, yd, z, zd)
  IMPLICIT NONE
  REAL, INTENT(IN) :: x, y
  REAL, INTENT(IN) :: xd, yd
  REAL, INTENT(OUT) :: z
  REAL, INTENT(OUT) :: zd
  zd = y*xd + x*yd
  z = x*y
END SUBROUTINE F_D
```

Running
```sh
$ tapenade g.f90
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Command: Took subroutine g as default differentiation root
Diff-liveness analysis turned off
@@ Created ./g_d.f90 
@@ Created ./g_d.msg
$ cat g_d.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of g in forward (tangent) mode:
!   variations   of useful results: v
!   with respect to varying inputs: z
!   RW status of diff variables: v:out z:in
SUBROUTINE G_D(z, zd, v, vd)
  IMPLICIT NONE
  REAL, INTENT(IN) :: z
  REAL, INTENT(IN) :: zd
  REAL, DIMENSION(2), INTENT(OUT) :: v
  REAL, DIMENSION(2), INTENT(OUT) :: vd
  INTRINSIC COS
  INTRINSIC SIN
  vd = (/COS(z)*zd, -(SIN(z)*zd)/)
  v = (/SIN(z), COS(z)/)
END SUBROUTINE G_D
```

</details>
</div>

## Verification: the Taylor test

Recall the Taylor expansion of a differentiable function $f:A\rightarrow\mathbb{R}$, for $A\subseteq\mathbb{R}$:
$$
    f(x+h)=f(x)+hf'(x) + \frac{1}{2!}h^2f''(x) + \dots + \frac{1}{n!}h^nf^{(n)}(x) + \dots,
$$
for some $h\in\mathbb{R}$, i.e.,
$$
    f(x+h)=f(x)+hf'(x) + \mathcal{O}(h^2).
$$
This gives rise to the first-order forward difference approximation of the first derivative as follows:
$$
    f'(x) \approx \frac{f(x+h)-f(x)}{h}.
$$

This is a rather crude approximation to a derivative, but it can do the job, given an appropriate choice of $h$.

The idea of the Taylor test is to take smaller and smaller spacing values $h$ from a given input value and to check that the difference between the forward difference and the AD-generated result converges quadratically. That is, we verify that
$$
    \|f(x+h)-f(x)-h\:\texttt{f\_d}(x)\|=\mathcal{O}(h^2),
$$
where $\texttt{f\_d}$ is the AD-generated result.

## $f$ \& $g$ example: Taylor test

We return to the previous example and double-check that we are happy with the output of the AD tool.

For simplicity of demonstration, we translate `g` and `g_d` into Python and conduct the Taylor test here.

<div class="alert alert-block alert-danger">
<b>TODO: Could we use f2py here?</b>    
</div>

In [ ]:
def g(z):
    """
    Python translation of the `g` subroutine from `session1/exercises/fg/g.f90` using NumPy.
    """
    return np.array([np.sin(z), np.cos(z)])

In [ ]:
def gd(z, zd):
    """
    Python translation of the `g_d` subroutine generated by Tapenade in `session1/exercises/fg/g_d.f90` using NumPy.
    """
    vd = np.array([np.cos(z) * zd, -np.sin(z) * zd])
    v = np.array([np.sin(z), np.cos(z)])
    return v, vd

In [ ]:
# Choose arbitrary input for g
z = 1.0

# Choose unit seed
zd = 1.0

# Compute the derivative using forward mode AD
_, gd_ad = gd(z, zd)

# Run the Taylor test over several spacing values
spacings = [1.0, 0.1, 0.01, 0.001]
errors = []
for h in spacings:
    errors.append(np.linalg.norm(g(z + h) - g(z) - h * gd_ad))

# Plot the solution, demonstrating that the expected quadratic convergence is achieved
fig, axes = plt.subplots()
axes.loglog(spacings, errors, "--x")
annotation.slope_marker((1e-2, 1e-4), 2, ax=axes, invert=True)
axes.grid()

<div class="alert alert-block alert-info">
<b>Optional exercise</b>
    
Perform the Taylor test to double-check you are happy with the AD-generated derivative of $f$, too. Note that the input will have two dimensions so you will have to consider perturbations in both directions separately.

</div>

## Example: ODE-constrained optimisation

Consider the scalar ordinary differential equation (ODE)
$$
    \frac{\mathrm{d}u}{\mathrm{d}t}=f(u),\quad u(0)=u_0,
$$
where $t\in[0,T]$ is the time variable, $T>0$ is the end time, and $u_0\in\mathbb{R}$ is the initial condition. Given some $f:A\rightarrow\mathbb{R}$ with $A\subseteq\mathbb{R}$, we seek to solve the ODE for $u:[0,T]\rightarrow\mathbb{R}$.

For simplicity, let's consider the ODE
$$
    \frac{\mathrm{d}u}{\mathrm{d}t}=u,\quad u(0)=1,
$$
i.e., $f(u)=u$.

<div class="alert alert-block alert-info">
<b>Optional exercise</b>
    
Convince yourself that the analytical solution of the ODE is $u=\mathrm{e}^t$.

<b>Solutions</b>
    
<details>

Plugging $u(t)=\mathrm{e}^t$ into the LHS gives $\frac{\mathrm{d}u}{\mathrm{d}t}=\mathrm{e}^t=u$, which satisfies the ODE. Checking the initial condition, we have $u(0)=\mathrm{e}^0=1$, which also satisfies.

</details>

## ODE example: forward vs backward Euler

You've probably aware of the simplest example of an explicit timestepping method to approximate the solution of the ODE is *forward Euler* (a.k.a. explicit Euler):
$$
    \frac{u_{k}-u_{k-1}}{\Delta t}=f(u_{k-1}),
$$
for $k\in\mathbb{N}$ and some timestep $\Delta t>0$.

You're probably also aware of the simplest example of an implicit timestepping method to approximate the solution of the ODE is *backward Euler* (a.k.a. implicit Euler):
$$
    \frac{u_{k}-u_{k-1}}{\Delta t}=f(u_k),
$$
for $k\in\mathbb{N}$ and some timestep $\Delta t>0$.

These are special cases of the first-order *theta-method*
$$
    \frac{u_{k}-u_{k-1}}{\Delta t}=(1-\theta)f(u_{k-1})+\theta f(u_k),
$$
where $\theta\in[0,1]$.

The forward and backward Euler approaches have been implemented for the case $f(u)=u$ in the `session1/exercises/ode` subdirectory.
```sh
$ cd exercises/ode
$ ls
backward_euler.f90 forward_euler.f90 theta.f90
```
Here `forward_euler.f90` and `backward_euler.f90` contain programs to solve the ODE, making use of common code from `theta.f90`, but with values $\theta=0$ and $\theta=1$, respectively.

<div class="alert alert-block alert-warning">
<b>Details on running the code</b>

<details>

Compile with
```sh
$ gfortran forward_euler.f90 -o forward
$ gfortran backward_euler.f90 -o backward
```
and then run with
```sh
$ ./forward
$ ./backward
$ ls
backward      backward_euler.f90  forward.csv        theta.f90
backward.csv  forward             forward_euler.f90
```

</details>
</div>

In [ ]:
df_forward = pandas.read_csv("exercises/ode/forward.csv")
df_backward = pandas.read_csv("exercises/ode/backward.csv")

fig, axes = plt.subplots()

times = np.linspace(0, 1, 101)

axes.plot(times, np.exp(times), "-", color="k", label="Analytical solution")
axes.plot(df_forward["t"], df_forward["u"], "--x", label="Forward Euler")
axes.plot(df_backward["t"], df_backward["u"], ":o", label="Backward Euler")
axes.legend()
axes.grid()

## ODE example: source transformation

As we see from the plot above, the forward Euler method tends to underestimate the solution, whereas the backward Euler method tends to overestimate it. Let's try to optimise the value of $\theta$ to best match the solution using a gradient-based optimisation method. To do that, we first need the gradient.

The optimisation problem we seek to solve is to minimise some error measure $J$ for the approximation of $u$ by varying $\theta$. That is,
$$
    \min_{\theta\in[0,1]}J(u;\theta).
$$
where the notation $J(u;\theta)$ refers to the implicit dependence of the solution approximation $u$ on $\theta$.

<div class="alert alert-block alert-info">
<b>Exercise</b>
    
Navigate to `session1/exercises/ode` and apply forward mode AD to the `theta_method` subroutine in `theta.f90` with Tapenade, specifying the `theta` argument as independent and the output `u` as dependent.

<b>Solutions</b>
    
<details>

```sh
$ tapenade -head "theta_method(theta)\(u)" theta.f90 
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Diff-liveness analysis turned off
@@ Created ./theta_d.f90 
@@ Created ./theta_d.msg
$ cat theta_d.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of theta_method in forward (tangent) mode:
!   variations   of useful results: u
!   with respect to varying inputs: theta
!   RW status of diff variables: u:out theta:in
SUBROUTINE THETA_METHOD_D(theta, thetad, filename, u, ud)
  IMPLICIT NONE
  REAL, INTENT(IN) :: theta
  REAL, INTENT(IN) :: thetad
  CHARACTER(len=*), INTENT(IN) :: filename
  REAL, INTENT(OUT) :: u
  REAL, INTENT(OUT) :: ud
  REAL, PARAMETER :: dt=0.1
  REAL, PARAMETER :: end_time=1.0
  REAL :: t
  REAL :: u_
  REAL :: u_d
! Create a CSV file for output
  OPEN(unit=10, file=filename) 
  WRITE(unit=10, fmt='(''t,u'')') 
! Initialisation
  t = 0.0
  CALL INITIAL_CONDITION(u_)
  WRITE(unit=10, fmt=100) t, u_
  ud = 0.0
  u_d = 0.0
! Timestepping loop
  DO WHILE (t .LT. end_time - 1e-05)
    CALL THETA_STEP_D(u, ud, u_, u_d, dt, theta, thetad)
    u_d = ud
    u_ = u
    t = t + dt
    WRITE(unit=10, fmt=100) t, u
  END DO
  CLOSE(unit=10) 
 100 FORMAT(f4.2,',',f4.2)
END SUBROUTINE THETA_METHOD_D

!  Differentiation of theta_step in forward (tangent) mode:
!   variations   of useful results: u
!   with respect to varying inputs: u_ theta
SUBROUTINE THETA_STEP_D(u, ud, u_, u_d, dt, theta, thetad)
  IMPLICIT NONE
! solution at current timestep
  REAL, INTENT(OUT) :: u
  REAL, INTENT(OUT) :: ud
! solution at previous timestep
  REAL, INTENT(IN) :: u_
  REAL, INTENT(IN) :: u_d
! Timestep length
  REAL, INTENT(IN) :: dt
! Theta parameter
  REAL, INTENT(IN) :: theta
  REAL, INTENT(IN) :: thetad
  REAL :: temp
  temp = u_/(-(dt*theta)+1)
  ud = (dt*(1-theta)+1)*(u_d+temp*dt*thetad)/(1-dt*theta) - temp*dt*&
&   thetad
  u = (dt*(1-theta)+1)*temp
END SUBROUTINE THETA_STEP_D
```

</details>

## ODE example: optimisation with gradient descent

Let's solve this ODE problem with one of the simplest gradient-based optimisation approaches: gradient descent. This amounts to an initial guess $\theta_0$, followed by iterative updates
$$
    \theta_{k+1}=\theta_k+\alpha_k\:p_k,
$$
where $\alpha_k$ is the step length at iteration $k$ and $p_k$ is the descent direction. For gradient descent, we simply take
$$
    p_k=-\frac{\mathrm{d}J_k}{\mathrm{d}\theta_k}.
$$

Since we know the analytical solution for this problem, we may make an 'artifical' choice of cost function such as
$$
    J(u;\theta)=(u(1)-\mathrm{e}^1)^2,
$$
where here $\mathrm{e}^1$ is the analytical solution at the end time $t=1$.

<div class="alert alert-block alert-info">
<b>Exercise</b>
    
The gradient descent algorithm is implemented for our ODE problem in `session1/exercises/ode/gradient_descent.f90`. However, there is another missing piece: you will also need to differentiate the cost function. Convince yourself you are satisfied with the output.

<b>Solutions</b>
    
<details>

```sh
$ tapenade cost_function.f90 
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Command: Took subroutine cost_function as default differentiation root
Diff-liveness analysis turned off
@@ Created ./cost_function_d.f90 
@@ Created ./cost_function_d.msg
cat cost_function_d.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of cost_function in forward (tangent) mode:
!   variations   of useful results: j
!   with respect to varying inputs: u
!   RW status of diff variables: j:out u:in
! Cost function evaluating the l2 error at the end time against the analytical solution u(t)=exp(t)
SUBROUTINE COST_FUNCTION_D(u, ud, j, jd)
  IMPLICIT NONE
  REAL, INTENT(IN) :: u
  REAL, INTENT(IN) :: ud
  REAL, INTENT(OUT) :: j
  REAL, INTENT(OUT) :: jd
  INTRINSIC EXP
  REAL, PARAMETER :: e=EXP(1.0)
  jd = 2*(u-e)*ud
  j = (u-e)**2
END SUBROUTINE COST_FUNCTION_D
```

<div class="alert alert-block alert-warning">
<b>Details on running the code</b>

<details>

Compile with
```sh
$ gfortran gradient_descent.f90 -o gradient_descent
```
and then run with
```sh
$ ./gradient_descent
$ ls
backward.csv        forward.csv           optimised.csv
backward_euler.f90  forward_euler.f90     theta.f90
cost_function.f90   gradient_descent.f90  theta_progress.csv
```

</details>
</div>

In [ ]:
df_opt = pandas.read_csv("exercises/ode/optimisation_progress.csv")

costs = np.array(df_opt["J"])
costs[0] = np.nan  # Remove the first entry because it's not actually a cost function value
controls = np.array(df_opt["theta"])

fig, axes = plt.subplots(ncols=2, figsize=(12, 5))
axes[0].loglog(df_opt["it"], costs, "--", label="Cost function value")
axes[0].legend()
axes[0].grid()
axes[1].plot(df_opt["it"], df_opt["theta"], "--", label="Control value")
axes[1].legend()
axes[1].grid()

In [ ]:
df_optimised = pandas.read_csv("exercises/ode/optimised.csv")

fig, axes = plt.subplots()
axes.plot(times, np.exp(times), "-", color="k", label="Analytical solution")
axes.plot(df_forward["t"], df_forward["u"], "--x", label="Forward Euler")
axes.plot(df_backward["t"], df_backward["u"], ":o", label="Backward Euler")
axes.plot(df_optimised["t"], df_optimised["u"], "-.^", label=rf"Optimised ($\theta={controls[-1]:.4f}$)")
axes.legend()
axes.grid()

As we might hope, the optimised value of $\theta$ gives a much better approximation.

## Calculating the *full* Jacobian

<div class="alert alert-block alert-info">
<b>Question</b>

Given a map $\mathbf{f}$, some input $\mathbf{x}$, and some seed $\dot{\mathbf{x}}$, we have the Jacobian vector product
$$\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}.$$

*How can we use this to compute the full Jacobian matrix $\nabla\mathbf{f}(\mathbf{x})$?*

<b>Solution</b>
    
<details>
$$\nabla F(x)=\nabla F(x)I_n=\nabla F(x)\begin{bmatrix}e_1,e_2,\dots,e_n\end{bmatrix}.$$

Apply JVP to the $n$ canonical unit vectors.
</details>
</div>

## Example: tridiagonal matrix

Consider the classic central difference approximation to the second derivative (yes, more derivatives!):
$$
    \frac{\mathrm{d}^2u}{\mathrm{d}x^2}\approx\frac{u(x-h)-2u(x)+u(x+h)}{h^2},
$$
for some uniform spacing $h>0$. For a uniform discretisation $\{x_k\}_{k=1}^n$ of an interval, this can be written as
$$
    \left.\frac{\mathrm{d}^2u}{\mathrm{d}x^2}\right|_{x=x_k}\approx\frac{u_{k-1}-2u_k+u_{k+1}}{h^2},
$$
where $u_k$ is an approximation of $u(x_k)$. This is implemented in Fortran in `session1/exercises/sparse/central_diff.f90` for the case of a periodic 1D function.

```fortran
! Central difference approximation for the second derivative of a periodic 1D function
subroutine central_diff(u, f, h, n)
  implicit none

  integer, intent(in) :: n
  real, intent(in), dimension(n) :: u
  real, intent(out), dimension(n) :: f
  real, intent(in) :: h

  integer :: i

  if (size(u, 1) /= n) then
    print *, "Invalid input array size"
    stop 1
  end if
  if (size(f, 1) /= n) then
    print *, "Invalid output array size"
    stop 1
  end if

  do i = 1, n
    if (i == 1) then
      f(i) = (u(n) - 2 * u(i) + u(i+1)) / h ** 2
    else if (i == n) then
      f(i) = (u(i-1) - 2 * u(i) + u(1)) / h ** 2
    else
      f(i) = (u(i-1) - 2 * u(i) + u(i+1)) / h ** 2
    end if
  end do
end subroutine central_diff
```

<div class="alert alert-block alert-info">
<b>Exercise</b>

Apply Tapenade to compute the derivative of the output array `f` with respect to the input array `u`.

<b>Solution</b>
    
<details>
    
Running
```sh
$ tapenade -head "central_diff(f)/(u)" central_diff.f90
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Diff-liveness analysis turned off
@@ Created ./central_diff_d.f90 
@@ Created ./central_diff_d.msg
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of central_diff in forward (tangent) mode:
!   variations   of useful results: f
!   with respect to varying inputs: u
!   RW status of diff variables: f:out u:in
! Central difference approximation for the second derivative of a periodic 1D function
SUBROUTINE CENTRAL_DIFF_D(u, ud, f, fd, h, n)
  IMPLICIT NONE
  INTEGER, INTENT(IN) :: n
  REAL, DIMENSION(n), INTENT(IN) :: u
  REAL, DIMENSION(n), INTENT(IN) :: ud
  REAL, DIMENSION(n), INTENT(OUT) :: f
  REAL, DIMENSION(n), INTENT(OUT) :: fd
  REAL, INTENT(IN) :: h
  INTEGER :: i
  INTRINSIC SIZE
  IF (SIZE(u, 1) .NE. n) THEN
    PRINT*, 'Invalid input array size'
    STOP
  ELSE IF (SIZE(f, 1) .NE. n) THEN
    PRINT*, 'Invalid output array size'
    STOP
  ELSE
    fd = 0.0
    DO i=1,n
      IF (i .EQ. 1) THEN
        fd(i) = (ud(n)-2*ud(i)+ud(i+1))/h**2
        f(i) = (u(n)-2*u(i)+u(i+1))/h**2
      ELSE IF (i .EQ. n) THEN
        fd(i) = (ud(i-1)-2*ud(i)+ud(1))/h**2
        f(i) = (u(i-1)-2*u(i)+u(1))/h**2
      ELSE
        fd(i) = (ud(i-1)-2*ud(i)+ud(i+1))/h**2
        f(i) = (u(i-1)-2*u(i)+u(i+1))/h**2
      END IF
    END DO
  END IF
END SUBROUTINE CENTRAL_DIFF_D
```

</details>
</div>

## Tridiagonal example: compute dense Jacobian

In `session1/exercises/sparse/dense_jacobian.f90` we provide a program for computing the Jacobian of the centred difference function over the uniform partition of the interval.

Running
```sh
$ cd session1/exercises/sparse
$ gfortran dense_jacobian.f90 -o dense_jacobian
$ ./dense_jacobian
```
produces a file `dense_jacobian.dat`, which we can read and view in the notebook. Note that the computation here required $n$ applications of forward mode AD, where $n$ is the number of gridpoints.

In [ ]:
lines = open("exercises/sparse/dense_jacobian.dat").readlines()
print(np.array([[float(entry) for entry in line[:-2].split(",")] for line in lines]))

## Sparse AD

We can compute the full Jacobian with
$$\nabla\mathbf{f}(\mathbf{x})=\nabla\mathbf{f}(\mathbf{x})I_n=\nabla\mathbf{f}(\mathbf{x})\begin{bmatrix}e_1,e_2,\dots,e_n\end{bmatrix}.$$

* But what about when $n$ gets very large?
* And what about when the Jacobian is sparse?

## Diagonal Jacobian example

Suppose $\nabla\mathbf{f}(\mathbf{x})$ is diagonal, say

$$\nabla\mathbf{f}(\mathbf{x})=\begin{bmatrix}f_1\\& f_2\\ & & \ddots\\ & & & f_n\end{bmatrix}.$$

Then, for a seed vector $\dot{\mathbf{x}}=\begin{bmatrix}\dot{x}_1 & \dot{x}_2 & \dots & \dot{x}_n\end{bmatrix}^T$, we have

$$
\nabla\mathbf{f}(\mathbf{x})\:\dot{\mathbf{x}}
=\begin{bmatrix}f_1\\& f_2\\ & & \ddots\\ & & & f_n\end{bmatrix}
\begin{bmatrix}\dot{x}_1 \\ \dot{x}_2 \\ \vdots \\ \dot{x}_n\end{bmatrix}
=\begin{bmatrix}f_1\dot{x}_1 \\ f_2\dot{x}_2 \\ \vdots \\ f_n\dot{x}_n\end{bmatrix}.
$$

Making the clever choice of seed vector $\dot{\mathbf{x}}:=\mathbf{e}=\begin{bmatrix}1 & 1 & \dots & 1\end{bmatrix}^T$, we can apply **a single JVP** and then back out the full Jacobian by putting each entry on the diagonal:
$$
    \mathrm{diag}(\nabla\mathbf{f}(\mathbf{x})\:\mathbf{e})
    =\mathrm{diag}\left(\begin{bmatrix}f_1 \\ f_2 \\ \vdots \\ f_n \end{bmatrix}\right)
    =\begin{bmatrix}f_1\\& f_2\\ & & \ddots\\ & & & f_n\end{bmatrix}
    =\nabla\mathbf{f}(\mathbf{x}).
$$

## Sparse AD: what colour is your Jacobian?

The example above is the simplest case of what is known as *colouring*. The idea is to group columns of the Jacobian such that the columns in each group are linearly independent. This gives rise to compression approaches for sparse AD.

<img src="images/colours.png" width=600 style="margin-left:auto; margin-right:auto"/>
<div><strong>Figure 2:</strong> Jacobian colouring diagram taken from Gebremedhin, et al (2005).</div>

## Tridiagonal example: compute compressed Jacobian

In `session1/exercises/sparse/compressed_jacobian.f90` we provide a program for computing the same Jacobian of the centred difference function but this time using a clever choice of seed vectors:
$$
    \begin{bmatrix}1\\0\\0\\1\\0\\0\\1\\0\\0\\1\end{bmatrix},
    \quad\begin{bmatrix}0\\1\\0\\0\\1\\0\\0\\1\\0\\0\end{bmatrix},
    \quad\begin{bmatrix}0\\0\\1\\0\\0\\1\\0\\0\\1\\0\end{bmatrix}.
$$

```fortran
include "central_diff_d.f90"

! Program for computing a tridiagonal Jacobian using compression
program compressed_jacobian
  implicit none

  integer, parameter :: n = 10
  real, parameter :: h = 1.0

  real, dimension(n) :: u, ud
  real, dimension(n) :: f, fd
  real, dimension(n,n) :: jacobian
  integer :: i, j

  ! Specify some arbitrary input
  u(:) = 1.0

  ! Set the seed vector to zero initially
  ud(:) = 0.0

  ! Loop over three offsets
  do i = 1, 3

    ! Apply the JVP to seed vectors based on the offsets
    ud(i::3) = 1.0
    call central_diff_d(u, ud, f, fd, h, n)
    ud(i::3) = 0.0

    ! Decompress rows and insert them into the Jacobian
    j = i
    jacobian(i::3,:) = 0.0
    do while (j <= n)
      if (j == 1) then
        jacobian(j,n) = fd(n)
      else
        jacobian(j,j-1) = fd(j-1)
      end if
      jacobian(j,j) = fd(j)
      if (j == n) then
        jacobian(j,1) = fd(1)
      else
        jacobian(j,j+1) = fd(j+1)
      end if
      j = j + 3
    end do
  end do

  ! Write out the result
  open(unit=10, file="compressed_jacobian.dat")
  do i = 1, n
    write(unit=10, fmt=100) jacobian(i,:)
  end do
  100 format(10(f4.1,","))
  close(unit=10)

end program compressed_jacobian
```

Running
```sh
$ cd session1/exercises/sparse
$ gfortran compressed_jacobian.f90 -o compressed_jacobian
$ ./compressed_jacobian
```
produces a file `compressed_jacobian.dat`, which we can again read and view in the notebook. This time we only need to compute **three** JVPs, rather than $n$!

In [ ]:
lines = open("exercises/sparse/compressed_jacobian.dat").readlines()
print(np.array([[float(entry) for entry in line[:-2].split(",")] for line in lines]))

## Sparse AD: further details

#### In practice

It's worth noting that we used knowledge about the tridiagonal structure of the Jacobian to choose the seed vectors... so that we could compute the Jacobian. For Jacobians in the wild (e.g., those related to networks) we don't necessarily have such intuition. In practice, we need to perform an initial step to establish the sparsity pattern before we can do the colouring.

#### Vector mode

Above, we applied the JVP to three different seed vectors. Most AD tools support a 'vector' mode, which allows this to be done in a single call. This is the *vector-Jacobian product (VJP)*:

$$\text{VJP}(\mathbf{f},\mathbf{x},\dot{\mathbf{X}}):=\begin{bmatrix}\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}_1,\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}_2,\dots,\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}_k\end{bmatrix},$$

for a *seed matrix* $\dot{\mathbf{X}}:=\begin{bmatrix}\dot{\mathbf{x}}_1,\dot{\mathbf{x}}_2,\dots,\dot{\mathbf{x}}_k\end{bmatrix}\in\mathbb{R}^{k\times n}$, $k\in\mathbb{N}$.

<div class="alert alert-block alert-info">
<b>Optional exercise</b>

1. Apply Tapenade to the `central_diff` subroutine.
2. Rework `session1/exercises/sparse/compressed_jacobian.f90` to use vector mode.

<b>Solution 1</b>
    
<details>
    
Running
```sh
$ cd session1/exercises/sparse
$ tapenade -head "central_diff(f)/(u)" -vector central_diff.f90
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
@@ Options:  multiDirectional
Diff-liveness analysis turned off
@@ Created ./central_diff_dv.f90 
@@ Created ./central_diff_dv.msg
$ cat central_diff_dv.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of central_diff in forward (tangent) mode (with options multiDirectional):
!   variations   of useful results: f
!   with respect to varying inputs: u
!   RW status of diff variables: f:out u:in
! Central difference approximation for the second derivative of a periodic 1D function
SUBROUTINE CENTRAL_DIFF_DV(u, ud, f, fd, h, n, nbdirs)
  USE DIFFSIZES
!  Hint: nbdirsmax should be the maximum number of differentiation directions
  IMPLICIT NONE
  INTEGER, INTENT(IN) :: n
  REAL, DIMENSION(n), INTENT(IN) :: u
  REAL, DIMENSION(nbdirsmax, n), INTENT(IN) :: ud
  REAL, DIMENSION(n), INTENT(OUT) :: f
  REAL, DIMENSION(nbdirsmax, n), INTENT(OUT) :: fd
  REAL, INTENT(IN) :: h
  INTEGER :: i
  INTRINSIC SIZE
  INTEGER :: nd
  INTEGER :: nbdirs
  IF (SIZE(u, 1) .NE. n) THEN
    PRINT*, 'Invalid input array size'
    STOP
  ELSE IF (SIZE(f, 1) .NE. n) THEN
    PRINT*, 'Invalid output array size'
    STOP
  ELSE
    fd = 0.0
    DO i=1,n
      IF (i .EQ. 1) THEN
        DO nd=1,nbdirs
          fd(nd, i) = (ud(nd, n)-2*ud(nd, i)+ud(nd, i+1))/h**2
        END DO
        f(i) = (u(n)-2*u(i)+u(i+1))/h**2
      ELSE IF (i .EQ. n) THEN
        DO nd=1,nbdirs
          fd(nd, i) = (ud(nd, i-1)-2*ud(nd, i)+ud(nd, 1))/h**2
        END DO
        f(i) = (u(i-1)-2*u(i)+u(1))/h**2
      ELSE
        DO nd=1,nbdirs
          fd(nd, i) = (ud(nd, i-1)-2*ud(nd, i)+ud(nd, i+1))/h**2
        END DO
        f(i) = (u(i-1)-2*u(i)+u(i+1))/h**2
      END IF
    END DO
  END IF
END SUBROUTINE CENTRAL_DIFF_DV
```

</details>

<br>
<b>Solution 2</b>
    
<details>

```fortran
! Module containing the nbdirsmax variable required by central_diff_dv
module diffsizes
  implicit none
  integer, parameter :: nbdirsmax = 3
end module diffsizes

! Include the vector forward mode derivative generated by Tapenade
include "central_diff_dv.f90"

! Program for computing a tridiagonal Jacobian using compression - vector version
program compressed_jacobian_vector
  implicit none

  integer, parameter :: n = 10
  real, parameter :: h = 1.0

  real, dimension(n) :: u, f
  real, dimension(3,n) :: udv, fdv
  real, dimension(n,n) :: jacobian
  integer :: i, j

  ! Specify some arbitrary input
  u(:) = 1.0

  ! Set the seed matrix
  udv(:,:) = 0.0
  do i = 1, 3
    udv(i,i::3) = 1.0
  end do

  ! Apply the VJP
  call central_diff_dv(u, udv, f, fdv, h, n, 3)

  ! Decompress rows and insert them into the Jacobian
  do i = 1, 3
    j = i
    jacobian(i::3,:) = 0.0
    do while (j <= n)
      if (j == 1) then
        jacobian(j,n) = fdv(i,n)
      else
        jacobian(j,j-1) = fdv(i,j-1)
      end if
      jacobian(j,j) = fdv(i,j)
      if (j == n) then
        jacobian(j,1) = fdv(i,1)
      else
        jacobian(j,j+1) = fdv(i,j+1)
      end if
      j = j + 3
    end do
  end do

  ! Write out the result
  open(unit=10, file="compressed_jacobian_vector.dat")
  do i = 1, n
    write(unit=10, fmt=100) jacobian(i,:)
  end do
  100 format(10(f4.1,","))
  close(unit=10)

end program compressed_jacobian_vector
```

</details>

</div>

## Sparse AD: overall performance comparison

We have considered three approaches for sparse AD:
1. 'Dense approach', which computes a JVP for each canonical unit vector.
2. 'Compressed approach', which computes JVPs for sums over sets of linearly independent columns.
3. 'Matrix-free approach', which applies a single JVP. (Only useful if the problem involves JVPs.)

In the following, we walk through a performance comparison of these different approaches applied to the solution of the nonlinear Gray-Scott equation
$$
    \frac{\partial u}{\partial t}=D_1\Delta u-uv^2+\gamma(1-u),
    \quad\frac{\partial v}{\partial t}=D_2\Delta v-uv^2=(\gamma+\kappa)v,
    \quad u,v:[0,2.5]^2\rightarrow\mathbb{R}
$$
(subject to some initial conditions) using a finite difference approach with Crank-Nicolson timestepping ($\theta=0.5$). This discretisation corresponds to a 5-point stencil and it turns out the Jacobian can be coloured with five colours. As such, the compressed approach involves five JVPs per Jacobian computation.

<img src="images/runtimes.png" width=600 style="margin-left:auto; margin-right:auto"/>
<div><strong>Figure 3:</strong> Performance comparison for different Jacobian computation approaches for solving the Gray-Scott equation, taken from Wallwork et al. (2019). PETSc was used for numerical solvers and ADOL-C for AD. The $N\times N$ notation in the subcaptions refers to the grid dimension.</div>

<!-- Run on a Knights Landing node at Argonne National Laboratory -->

<br>

Here 'analytic' refers to the hand-derived Jacobian, which we can't realistically expect to beat.

<!-- Notes

* Compressed approach exhibits strong scalability.

-->

## Sparse AD: convergence comparison

Matrix-free works well for small dimensions. However, it slows down at higher resolution because Jacobian assembly is done at each iteration of the *nonlinear* solver, whereas the JVP is calculated at each iteration of the *linear* solver. It's possible that a different choice of linear solver or preconditioner would reduce the cost of the matrix-free approach.

<img src="images/linear_solver_table.png" width=600 style="margin-left:auto; margin-right:auto"/>
<div><strong>Figure 4:</strong> Linear solver iterations as a function of grid resolution and number of processors for the Gray-Scott problem. Taken from Wallwork et al. (2019).</div>

## Sparse AD: performance comparison between steps

As it turns out, the cost of the sparsity detection and colouring are negligible for this particular setup. The cost is dominated by the decompression (recovery), with the propagation of seed vectors the next-most expensive step.

<img src="images/sparse_ad_proportions.png" width=600 style="margin-left:auto; margin-right:auto"/>
<div><strong>Figure 5:</strong> Comparative runtimes of components of the "compressed" sparse AD approach for the Gray-Scott problem. Taken from Wallwork et al. (2019).</div>

## Summary and outlook

In today's session we:
* Got a brief history of automatic differentiation.
* Learnt about *forward mode* and the *source transformation* approach.
* Tried out the *Tapenade* source transformation AD tool applied to some test problems.
* Verified the code generated by Tapenade both manually and using the *Taylor test*.
* Learnt about compressed and matrix-free approaches for sparse problems.
* Considered how different levels of abstraction can be appropriate for different problems.

In tomorrow's session (at the same time) we will:
* Learn about *reverse mode* and the *operator overloading* approach, comparing them with forward mode and source transformation, respectively.
* Verify the consistency of code generated by Tapenade under forward and reverse mode using the *dot product test*.
* Try out the *autograd* operator overloading AD tool underpinnning PyTorch (as well as libtorch (C++) and FTorch (Fortran)).
* Learn about *checkpointing* and using AD to compute higher order derivatives.
* See a showcase of some more advanced use cases of AD using Firedrake.

## References

* R. E. Wengert. *A simple automatic derivative evaluation program*. Communications
of the ACM, 7(8):463–464, 1964.
* A. Griewank. *Achieving logarithmic growth of temporal and spatial complexity in
reverse automatic differentiation.* Optimization Methods & Software, 1:35–54, 1992.
* D. Cortild, et al. *A Brief Review of Automatic Differentiation.* (2023).
* A. H. Gebremedhin, et al. *What color is your Jacobian? Graph coloring for computing derivatives* (2005). SIAM review, 47(4), pp.629-705.
* J. G. Wallwork, et al. *Computing derivatives for petsc adjoint solvers using algorithmic differentiation* (2019), arXiv preprint arXiv:1909.02836.